In [ ]:
using Revise
using Bilevel

using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using ForwardDiff
using DiffResults

using Bilevel: snopt, contact_τ, generate_autodiff_solver_fn, contact_jacobian!, auglag, svd

using BenchmarkTools
using ProfileView

In [ ]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, 1.)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles)
Δt = .005

In [ ]:
sim_data = get_sim_data_direct(mechanism,env,Δt)

In [ ]:
function normal_τ!(τ,sim_data::SimData,Hi,envj::EnvironmentJacobian,dyn_bias,u0,v0,x_upper::AbstractArray{U},n::Int) where U
    num_contacts = length(sim_data.env.contacts)
    env = sim_data.env
    lower_vs = sim_data.normal_vs[n]
    lower_cs = sim_data.normal_cs[n]
    lower_options = sim_data.normal_options[n]
    h = sim_data.Δt

    ϕAs = []
    ϕbs = []
    for i = 1:num_contacts
        J = envj.contact_jacobians[i].J
        ϕ = envj.contact_jacobians[i].ϕ
        N = envj.contact_jacobians[i].N

        ϕA = h^2*N*Hi*J
        ϕb = N*(h^2*Hi*(dyn_bias - u0) .- h*v0) .- ϕ

        push!(ϕAs,ϕA)
        push!(ϕbs,ϕb)
    end

    function eval_obj_(x::AbstractArray{L}) where L
        obj = 0.

        for i = 1:num_contacts
            c_n = lower_vs(x, Symbol("c_n", i))
            obj += c_n'*c_n
        end

        obj
    end

    function eval_cons_(x::AbstractArray{L}) where L
        # g = zeros(ForwardDiff.Dual, lower_cs.num_eqs + lower_cs.num_ineqs)
        # g = zeros(Real, lower_cs.num_eqs + lower_cs.num_ineqs)
        g = []

        for i = 1:num_contacts
            c_n = lower_vs(x, Symbol("c_n", i))
            # g[lower_cs(Symbol("c_n_pos", i))] .= -c_n
            # g[lower_cs(Symbol("ϕ", i))] .= ϕAs[i]*vcat(c_n, zeros(4)) + ϕbs[i] # TODO use β_dim
            g = vcat(g, -c_n)
            g = vcat(g, ϕAs[i]*vcat(c_n, zeros(4)) + ϕbs[i])
        end

        g
    end

    fres = DiffResults.HessianResult(zeros(U, lower_vs.num_vars))
    gres = DiffResults.JacobianResult(zeros(U, lower_cs.num_cons), zeros(U, lower_vs.num_vars))
    solver_fn_ = generate_autodiff_solver_fn(eval_obj_,fres,eval_cons_,gres,lower_cs.eqs,lower_cs.ineqs)
    # solver_fn_ = generate_autodiff_solver_fn(eval_obj_,eval_cons_,lower_cs.eqs,lower_cs.ineqs)

    x0 = zeros(lower_vs.num_vars)

    xopt, info = auglag(solver_fn_, lower_cs.num_eqs, lower_cs.num_ineqs, x0, lower_options)

    # TODO include the total weight here, not in J
    τ .= mapreduce(+, enumerate(envj.contact_jacobians)) do (i,cj)
        contact_τ(cj, lower_vs(xopt, Symbol("c_n", i)), zeros(4))
    end
    
#     display("normal")
#     solver_fn_snopt = generate_autodiff_solver_fn(eval_obj_,eval_cons_,lower_cs.eqs,lower_cs.ineqs)
#     options_snopt = Dict{String, Any}()
#     options_snopt["Derivative option"] = 1
#     options_snopt["Verify level"] = -1 # -1 => 0ff, 0 => cheap
#     xopt_snopt, info_snopt = snopt(solver_fn_snopt, lower_cs.num_eqs, lower_cs.num_ineqs, x0, options_snopt)
#     display(info_snopt)
#     τ_snopt = zeros(length(τ))
#     τ_snopt .= mapreduce(+, enumerate(envj.contact_jacobians)) do (i,cj)
#         contact_τ(cj, lower_vs(xopt_snopt, Symbol("c_n", i)), zeros(4))
#     end
#     display("snopt")
#     display(xopt_snopt)
#     display(τ_snopt)
#     display("auglag")
#     display(xopt)
#     display(τ)
    
    xopt
end

function friction_τ!(τ,sim_data::SimData,Hi,envj::EnvironmentJacobian,dyn_bias,u0,v0,x_upper::AbstractArray{U},x_normal::AbstractArray{N},n::Int) where {U,N}
    num_contacts = length(sim_data.env.contacts)
    env = sim_data.env
    normal_vs = sim_data.normal_vs[n]
    lower_vs = sim_data.fric_vs[n]
    lower_cs = sim_data.fric_cs[n]
    lower_options = sim_data.fric_options[n]
    h = sim_data.Δt
    total_weight = mass(sim_data.mechanism) * norm(sim_data.mechanism.gravitational_acceleration)
    
    Qds = []
    rds = []
    for i = 1:num_contacts
        J = envj.contact_jacobians[i].J
        ϕ = envj.contact_jacobians[i].ϕ
        N = envj.contact_jacobians[i].N

        Qd = h^2*J'*Hi*J
        rd = J'*(h^2*Hi*(dyn_bias - u0) .- h*v0)

        push!(Qds,Qd)
        push!(rds,rd)
    end
    
    function eval_obj_(x::AbstractArray{L}) where L
        obj = 0.

        for i = 1:num_contacts
            
            c_n = normal_vs(x_normal, Symbol("c_n", i))
            β = lower_vs(x, Symbol("β", i))
            z = vcat(c_n, β)
            obj += .5*z'*Qds[i]*z + z'*rds[i]
        end

        obj
    end

    function eval_cons_(x::AbstractArray{L}) where L
        # g = zeros(ForwardDiff.Dual, lower_cs.num_eqs + lower_cs.num_ineqs)
        # g = zeros(Real, lower_cs.num_eqs + lower_cs.num_ineqs)
        g = []

        for i = 1:num_contacts
            c_n = normal_vs(x_normal, Symbol("c_n", i))
            β = lower_vs(x, Symbol("β", i))
            # g[lower_cs(Symbol("β_pos", i))] .= -β
            # g[lower_cs(Symbol("fric_cone", i))] .= sum(β) .- env.contacts[i].obstacle.μ * c_n
            g = vcat(g, -β)
            g = vcat(g, sum(β) .- env.contacts[i].obstacle.μ * c_n)
        end

        g
    end

    fres = DiffResults.HessianResult(zeros(U, lower_vs.num_vars))
    gres = DiffResults.JacobianResult(zeros(U, lower_cs.num_cons), zeros(U, lower_vs.num_vars))
    solver_fn_ = generate_autodiff_solver_fn(eval_obj_,fres,eval_cons_,gres,lower_cs.eqs,lower_cs.ineqs)
    # solver_fn_ = generate_autodiff_solver_fn(eval_obj_,eval_cons_,lower_cs.eqs,lower_cs.ineqs)

    x0 = zeros(lower_vs.num_vars)

    xopt, info = auglag(solver_fn_, lower_cs.num_eqs, lower_cs.num_ineqs, x0, lower_options)

    # TODO include the total weight here, not in J
    τ .= mapreduce(+, enumerate(envj.contact_jacobians)) do (i,cj)
        contact_τ(cj, normal_vs(x_normal, Symbol("c_n", i)), lower_vs(xopt, Symbol("β", i)))
    end
    
#     display("friction")
#     solver_fn_snopt = generate_autodiff_solver_fn(eval_obj_,eval_cons_,lower_cs.eqs,lower_cs.ineqs)
#     options_snopt = Dict{String, Any}()
#     options_snopt["Derivative option"] = 1
#     options_snopt["Verify level"] = -1 # -1 => 0ff, 0 => cheap
#     xopt_snopt, info_snopt = snopt(solver_fn_snopt, lower_cs.num_eqs, lower_cs.num_ineqs, x0, options_snopt)
#     display(info_snopt)
#     τ_snopt = zeros(length(τ))
#     τ_snopt .= mapreduce(+, enumerate(envj.contact_jacobians)) do (i,cj)
#         contact_τ(cj, normal_vs(x_normal, Symbol("c_n", i)), lower_vs(xopt, Symbol("β", i)))
#     end
#     display("snopt")
#     display(xopt_snopt)
#     display(τ_snopt)
#     display("auglag")
#     display(xopt)
#     display(τ)

    xopt
end

function generate_solver_fn(sim_data,q0,v0,u0)
    x0 = sim_data.state_cache[1][Float64]
    envj = sim_data.envj_cache[1][Float64]
    Δt = sim_data.Δt
    vs = sim_data.vs
    cs = sim_data.cs

    relax_comp = haskey(vs.vars, :slack)
    num_contacts = length(sim_data.env.contacts)
    num_vel = num_velocities(sim_data.mechanism)

    set_configuration!(x0, q0)
    set_velocity!(x0, v0)
    setdirty!(x0)
    H = mass_matrix(x0)
    Hi = inv(H)

    contact_jacobian!(envj, x0)
    dyn_bias0 = dynamics_bias(x0) # TODO preallocate

    function eval_cons(x::AbstractArray{T}) where T
        xn = sim_data.state_cache[2][T]

        normal_bias = Vector{T}(undef, num_vel)
        contact_bias = Vector{T}(undef, num_vel)
        g = Vector{T}(undef, cs.num_eqs + cs.num_ineqs) # TODO preallocate

        qnext = vs(x, :qnext)
        vnext = vs(x, :vnext)
        if relax_comp
            slack = vs(x, :slack)
        end

        set_configuration!(xn, qnext)
        set_velocity!(xn, vnext)
        setdirty!(xn)

        if (num_contacts > 0)
            # compute normal forces
            x_normal = normal_τ!(normal_bias, sim_data, Hi, envj, dyn_bias0, u0, v0, x, 1)

            # compute friction forces
            friction_τ!(contact_bias, sim_data, Hi, envj, dyn_bias0, u0, v0, x, x_normal, 1)
        end
        config_derivative = configuration_derivative(xn) # TODO preallocate
        dyn_bias = dynamics_bias(xn) # TODO preallocate

        g[cs(:kin)] .= qnext .- q0 .- Δt .* config_derivative
        g[cs(:dyn)] .= H * (vnext - v0) .- Δt .* (u0 .- dyn_bias .- contact_bias)

        g
    end

    eval_cons
end

In [ ]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 1., 0., 0.]
u0 = [0., 0., 0., 0., 0., 0.]

sim_data.normal_options[1]["num_fosteps"] = 1
sim_data.normal_options[1]["num_sosteps"] = 15
sim_data.normal_options[1]["c"] = 1
sim_data.normal_options[1]["c_fos"] = 1
sim_data.normal_options[1]["c_sos"] = 1

sim_data.fric_options[1]["num_fosteps"] = 1
sim_data.fric_options[1]["num_sosteps"] = 15
sim_data.fric_options[1]["c"] = 1
sim_data.fric_options[1]["c_fos"] = 1
sim_data.fric_options[1]["c_sos"] = 1

eval_cons = generate_solver_fn(sim_data,q0,v0,u0)

In [ ]:
x = vcat(q0,v0)
sol = eval_cons(x)

In [ ]:
J = ForwardDiff.jacobian(eval_cons, x)

In [ ]:
ϵ = sqrt(eps(1.))
J_num = zeros(size(J))
for i = 1:length(x)
    δ = zeros(length(x))
    δ[i] = ϵ
    J_num[:,i] = (eval_cons(x + δ) .- sol)/ϵ
end
J_num

In [ ]:
maximum(abs.(J_num - J))

# SVD grad performance

In [1]:
using Revise
using Bilevel

using LinearAlgebra
using ForwardDiff

using Bilevel: snopt, contact_τ, generate_autodiff_solver_fn, contact_jacobian!, auglag, svd

using BenchmarkTools
using ProfileView
using Profile

┌ Info: Precompiling Bilevel [9688c538-179f-11e9-3174-495cea6b7f67]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file /home/blandry/.julia/compiled/v1.1/MechanismGeometries/lrgai.ji for MechanismGeometries [931e9471-e8fb-5385-a477-07ad12718aca]
└ @ Base loading.jl:1184
┌ Error: evaluation error
│   mod = Bilevel
│   ex = const usrfun = #= /home/blandry/.julia/dev/Bilevel/src/solvers/snopt.jl:154 =# @cfunction(objcon_wrapper, Cvoid, (Ptr{Clong}, Ref{Clong}, Ptr{Cdouble}, Ref{Clong}, Ref{Clong}, Ptr{Cdouble}, Ref{Clong}, Ref{Clong}, Ptr{Cdouble}, Ptr{Cchar}, Ref{Clong}, Ptr{Clong}, Ref{Clong}, Ptr{Cdouble}, Ref{Clong}))
│   exception = (ErrorException("invalid lookup expr (\$(QuoteNode(Core.svec)))(JuliaInterpreter.SSAValue(2), JuliaInterpreter.SSAValue(3), JuliaInterpreter.SSAValue(4), JuliaInterpreter.SSAValue(5), JuliaInterpreter.SSAValue(6), JuliaInterpreter.SSAValue(7), JuliaInterpreter.SSAValue(8), JuliaInterpreter.SSAValue(9), JuliaInterpreter.SSAValue(10), JuliaInterp

In [10]:
A = rand(6,6)
d = svd(A)

# d.S[3] = d.S[2]
# d.S[4] = d.S[5]
# d.S[10] = d.S[9]
# d.S[6:10] .= d.S[5]

A = d.U*Diagonal(d.S)*d.V'
z0 = A[:]

function f(a)
    n = Int(sqrt(length(a)))
    A = reshape(a,n,n) .* 2.
    U,s,V = svd(A)
    return vcat(U[:],s[:],V[:])
end

f (generic function with 1 method)

In [12]:
@benchmark J_auto = ForwardDiff.jacobian(f,z0)

BenchmarkTools.Trial: 
  memory estimate:  1.35 GiB
  allocs estimate:  21118
  --------------
  minimum time:     18.925 s (7.14% GC)
  median time:      18.925 s (7.14% GC)
  mean time:        18.925 s (7.14% GC)
  maximum time:     18.925 s (7.14% GC)
  --------------
  samples:          1
  evals/sample:     1

In [ ]:
@benchmark J_auto = ForwardDiff.jacobian(f,z0)

In [ ]:
Profile.clear()
@profile J_auto = ForwardDiff.jacobian(f,z0)

In [ ]:
ProfileView.view()